In [ ]:
!pip install fuzzywuzzy
!pip install Levenshtein
!pip install nltk

In [ ]:
import numpy as np
import regex as re
import pandas as pd
import spacy
import nltk
nltk.download("book")

lire les fichiers


In [ ]:
data = pd.read_csv("scrapped_data_from_NVS_P07+A05.csv")
data_en = pd.read_csv("terms_EN_Sabrine.csv")

créer une liste des termes standardisés des ressources

In [ ]:
L_ressource=[]
dt=data.to_numpy()
L_ressource = list(dt[0])
print(L_ressource)

créer une liste des termes candidats


In [ ]:
L_BioTex= list(data_en['term'])
print(L_BioTex)

nettoyage de la liste des termes candidats

In [ ]:
def contains_num(s):
  for i in s:
    if i.isnumeric():
      return True
  return False
contains_num("1000-m argo")
#s='18km sampling'
#s.isalpha()

In [ ]:
def clean_num(s):
  l=s.split(" ")
  for i in l:
    if contains_num(i):
      l.remove(i)
  return  " ".join(l)
clean_num('0–2000-m')


In [ ]:
def net(s):
  l=s.split()
  if len(l)>1:
    if len(l[0])>len(l[1]):
      return(l[0])
    else:
      return(l[1])
  else:
    return(s)

In [ ]:
L_BioTex.remove(np.nan)
L_BioTex.remove(np.nan)

In [ ]:
l=[i for i in L_BioTex if contains_num(i) and len(i)>3]
l=[re.sub("\d+", "", i) for i in l]
l=[net(i) for i in l]
l=[i.replace("-"," ") for i in l]
l=[i for i in l if len(i)>3]
l.remove("b ax")
list(set(l))

In [ ]:
clean=[clean_num(i) for i in L_BioTex if contains_num(i)==False]
clean=[i for i in clean if i!=None]

In [ ]:
clean+=l
len(clean)
clean=[i for i in clean if len(i)>3]

clean

nettoyage de la liste des termes standardisés

In [ ]:
L_ressource =[str(v).replace("_"," ") for v in L_ressource]
 L_ressource

**ANALYSE LEXICALE**

Application de fuzzywuzzy entre les 100 permiers termes candidats issus des fiches de métadonnées de catalogues ODATIS et les termes standardisés issus des ressources terminologiques NERC afin de calculer la mesure de similarité

In [ ]:
from fuzzywuzzy import process, fuzz

tab=np.empty
tab=['terme issue de Biotex','terme issue de ressources','Ratios']
df = pd.DataFrame(columns=tab)
s = 0
for i in range((len(clean)) :
  str2Match = clean[i]
  print("pour le terme : ", clean[i])
  str0=[]
  for j in range (len(L_ressource)) :
    str0.append(L_ressource[j]) 
  strOptions = str0
  Ratios = process.extract(str2Match,strOptions, limit=100)
  print(Ratios)
  for k in range(len(Ratios)):
    L = [clean[i], Ratios[k][0], Ratios[k][1]]
    df.loc[s]=L
    s = s + 1
  # You can also select the string with the highest matching percentage
  #highest = process.extractOne(str2Match,strOptions)
  #print(highest)

In [ ]:
df.to_excel('fuzz_B100_R100.xlsx')

In [ ]:
df

évaluation des resultats 

In [ ]:
# pour un seuil supérieur ou égal à 50%
df2=df.loc[df['Ratios']>=50 ]
df2
df2.to_excel('fuzz_B100_100_50.xlsx')

In [ ]:
# pour un seuil supérieur ou égal à 60%
df2=df.loc[df['Ratios']>=60 ]
df2
df2.to_excel('fuzz_B100_100_60.xlsx')

In [ ]:
# pour un seuil supérieur ou égal à 70%
df2=df.loc[df['Ratios']>=70 ]
df2
df2.to_excel('fuzz_B100_100_80.xlsx')

In [ ]:
# pour un seuil supérieur ou égal à 80%
df2=df.loc[df['Ratios']>=80 ]
df2
df2.to_excel('fuzz_B100_100_80.xlsx')

In [ ]:
## pour un seuil supérieur ou égal à 90%
df2=df.loc[df['Ratios']>=90 ]
df2
df2.to_excel('fuzz_B100_100_90.xlsx')

Application de Levenshtein entre les 100 permiers termes candidats issus des fiches de métadonnées de catalogues ODATIS et les termes standardisés issus des ressources terminologiques NERC afin de calculer la mesure de similarité

In [ ]:
tab1=np.empty
tab1=['terme issue de Biotex','terme issue de ressources','Ratios']
df1 = pd.DataFrame(columns=tab1)
s1 = 0
import Levenshtein as lev
def levCalclulate(str1, str2):
    Distance = lev.distance(str1, str2)
    #Ratio = 1 - lev.distance(str1, str2) / max(len(str1), len(str2))
    Ratio = lev.ratio(str1, str2)
    print("Levenshtein entre {0} et {1}".format(str1, str2))
    print("> Distance: {0}\n> Ratio: {1}\n".format(Distance, Ratio))
    return Ratio

for i in range(100) :
#for i in range(len(L_ressource)) :
  str1 = clean[i]
  print("pour le terme : ", clean[i])
  for j in range(len(L_ressource)):
    str2= L_ressource[j]
    Ratios = levCalclulate(str1, str2)
    print(Ratios)
    L1 = [clean[i],L_ressource[j], Ratios]
    df1.loc[s1]=L1
    s1 = s1 + 1
#df1.to_excel('lev_Ratio_B100_R(A05+P07).xlsx')

In [ ]:
df1.to_excel('lev_ratio_B100_R100.xlsx')

In [ ]:
df1

évaluation des résultats 

In [ ]:
## pour un seuil supérieur ou égal à 50%
df22=df1.loc[df1['Ratios']>=0.5 ]
df22
df22.to_excel('lev_ratio_B100_R100_50.xlsx')

In [ ]:
## pour un seuil supérieur ou égal à 60%
df22=df1.loc[df1['Ratios']>=0.6 ]
df22
df22.to_excel('lev_ratio_B100_R100_60.xlsx')

In [ ]:
## pour un seuil supérieur ou égal à 70%
df22=df1.loc[df1['Ratios']>=0.7]
df22
df22.to_excel('lev_ratio_B100_R100_70.xlsx')

In [ ]:
## pour un seuil supérieur ou égal à 80%
df22=df1.loc[df1['Ratios']>=0.8 ]
df22
df22.to_excel('lev_ratio_B100_R100_80.xlsx')

**ANALYSE SYNTAXIQUE** 

In [ ]:
from nltk import pos_tag

In [ ]:
s=[sentences.split() for sentences in clean]
s

In [ ]:
#v=[sentences.split() for sentences in L_ressource]
#v

In [ ]:
s1=[pos_tag(sent1) for sent1 in s]
s1

In [ ]:
#v1=[pos_tag(sent) for sent in v]
#v1

In [ ]:
#list1=eval(str(v1).replace("(","").replace(")",""))
list2=eval(str(s1).replace("(","").replace(")",""))

In [ ]:
#dt1=pd.DataFrame(list1)
#df.info()
dt2=pd.DataFrame(list2)

In [ ]:
#dt1.columns=['Col1','Col2','Col3','Col4','Col5','Col6','Col7','Col8'] 
dt2.columns=['Col1','Col2','Col3','Col4'] 

In [ ]:
dt1['terme issus de ressources']=dt1['Col1']+" "+dt1['Col3']+" "+dt1['Col5']+" "+dt1['Col7']
dt1['patron']=dt1['Col2']+" "+dt1['Col4']+" "+dt1['Col6']+" "+dt1['Col8']
#dt1.fillna("",inplace=True)
dt1['tête']=""
dt1


Identifier les patrons syntaxiques

In [ ]:
dt2['terme issus de Biotex']=dt2['Col1']+" "+dt2['Col3']
dt2['patron']=dt2['Col2']+" "+dt2['Col4']
#dt2.fillna("",inplace=True)
dt2

Identifier les combinaisons

In [ ]:
#dt1.partron.unique().tolist()
dt2.partron.unique().tolist()

In [ ]:
def transform(df):
  if (df['patron']=='NN ') or (df['patron']=='NNS ')or (df['patron']=='NNP ') or (df['patron']=='NN VBG') or (df['patron']=='VBG ') or (df['patron']=='JJ ') or (df['patron']=='NNS VBP')or (df['patron']=='VBG ') or (df['patron']=='JJ ') or (df['patron']=='NNS VBP ') :
    return df['Col1']
  if (df['patron']=='NN NN')or (df['patron']=='NNS NNS') or (df['patron']=='NN NNS')or (df['patron']=='NN NNP ') or (df['patron']=='NNS NN') or (df['patron']=='NNP NN') or (df['patron']=='JJ NN') or (df['patron']=='JJ NNS') or (df['patron']=='JJ NNP') or (df['patron']=='VBN NN') or (df['patron']=='VBG NNS')or (df['patron']=='VBG NN') or (df['patron']=='RB NNS')or (df['patron']=='RB NNP')or (df['patron']=='RB JJ')or (df['patron']=='RB NN'):
    return df['Col3']
 

In [ ]:
def transform1(df):
  if (df['partron']=='NN   ') or (df['partron']=='NNP NN CD ')or (df['partron']=='NNP CC NN ') or (df['partron']=='NNS   ') or (df['partron']=='NNP JJ NN ')or (df['partron']=='NNP JJ NN ')or (df['partron']=='NNP NN CC NN')or (df['partron']=='NN NN  ') :                                                                                                                         
    return df['Col1']
  if (df['partron']=='NNP NN  ') or (df['partron']=='JJ NNS  ')or (df['partron']=='JJ NN  ')or (df['partron']=='NNP NNP  ')or (df['partron']=='JJ NNS  ')or (df['partron']=='VBN NNS  ')or (df['partron']=='IN NN  '):
    return df['Col3']
  if (df['partron']=='NN VBG') or (df['partron']=='VBG   ') or (df['partron']=='JJ   ') or (df['partron']=='NNS VBP') or (df['partron']=='RB   ') or (df['partron']=='VB   ')or (df['partron']=='NN NNS  '):
    return df['Col1']

Spécifier les têtes

In [ ]:
#dt1['tête'] = dt2.apply(transform1, axis=1)

dt2['tête'] = dt2.apply(transform, axis=1)


In [ ]:
#dt1

In [ ]:
dt2

In [ ]:
#dt1.to_csv('Tête_R.CSV')
#dt1.to_excel('Tête_R.xlsx')
dt2.to_csv('Tête_B.CSV')
dt2.to_excel('Tête_B.xlsx')